## Loading in and Preprocessing the Dataset

In [1]:
#importing necessary libraries
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_predict
import numpy as np
from scikeras.wrappers import KerasClassifier

In [2]:
#loading in and rescaling the data

train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Create ImageDataGenerators with rescaling
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary'
)
val_generator = val_datagen.flow_from_directory(
    val_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary'
)

# Print the class indices
print("Class indices:", train_generator.class_indices)


Found 3616 images belonging to 2 classes.
Found 1616 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Class indices: {'NORMAL': 0, 'PNEUMONIA': 1}


In [6]:
# Extracting data from generators to use with cross_val_predict
def extract_data(generator):
    X, y = [], []
    # Iterate over the generator to get all batches
    for _ in range(len(generator)):
        x_batch, y_batch = generator.next()  # Get the next batch of images and labels
        X.extend(x_batch)  # Append images to the list
        y.extend(y_batch)  # Append labels to the list
    return np.array(X), np.array(y)  # Convert lists to NumPy arrays

# Extract data and labels from the training generator
X_train_all, y_train_scaled = extract_data(train_generator)


## Testing a Baseline Neural Network

In [12]:
from tensorflow.keras import models, layers, regularizers
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Define the input shape
input_shape = (150, 150, 3)

# Create the baseline model
baseline_model = models.Sequential()
baseline_model.add(layers.Flatten(input_shape=input_shape))
baseline_model.add(layers.Dense(512, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the baseline model on the training data
baseline_model.fit(train_generator, epochs=50, validation_data=val_generator)

# Evaluate the baseline model on the validation data
val_loss, val_accuracy = baseline_model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Get predictions for the validation data
val_preds = baseline_model.predict(val_generator)
val_preds = (val_preds > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

# Calculate additional metrics
val_precision = precision_score(val_true, val_preds)
val_recall = recall_score(val_true, val_preds)
val_f1 = f1_score(val_true, val_preds)
val_conf_matrix = confusion_matrix(val_true, val_preds)

print("Baseline Model Validation Metrics:")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")
print(f"Confusion Matrix:\n {val_conf_matrix}")


Epoch 1/50
113/113 [==============================] - 144s 1s/step - loss: 5.2112 - accuracy: 0.8526 - val_loss: 3.5199 - val_accuracy: 0.6937
Epoch 2/50
113/113 [==============================] - 102s 901ms/step - loss: 0.7617 - accuracy: 0.9287 - val_loss: 0.6487 - val_accuracy: 0.9140
Epoch 3/50
113/113 [==============================] - 112s 994ms/step - loss: 0.3179 - accuracy: 0.9477 - val_loss: 1.1328 - val_accuracy: 0.8391
Epoch 4/50
113/113 [==============================] - 103s 909ms/step - loss: 0.6222 - accuracy: 0.9300 - val_loss: 0.8589 - val_accuracy: 0.8998
Epoch 5/50
113/113 [==============================] - 98s 868ms/step - loss: 0.4384 - accuracy: 0.9433 - val_loss: 0.5845 - val_accuracy: 0.9196
Epoch 6/50
113/113 [==============================] - 96s 852ms/step - loss: 0.3862 - accuracy: 0.9477 - val_loss: 1.6925 - val_accuracy: 0.8063
Epoch 7/50
113/113 [==============================] - 98s 864ms/step - loss: 0.3542 - accuracy: 0.9452 - val_loss: 1.2780 - val_a

## Adding in Early Stopping

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the input shape
input_shape = (150, 150, 3)

# Create the baseline model
baseline_model = models.Sequential()
baseline_model.add(layers.Flatten(input_shape=input_shape))
baseline_model.add(layers.Dense(512, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the baseline model with early stopping
history = baseline_model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the baseline model on the validation data
val_loss, val_accuracy = baseline_model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Get predictions for the validation data
val_preds = baseline_model.predict(val_generator)
val_preds = (val_preds > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

# Calculate additional metrics
val_precision = precision_score(val_true, val_preds)
val_recall = recall_score(val_true, val_preds)
val_f1 = f1_score(val_true, val_preds)
val_conf_matrix = confusion_matrix(val_true, val_preds)

print("Baseline Model Validation Metrics:")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")
print(f"Confusion Matrix:\n {val_conf_matrix}")


Epoch 1/50
113/113 [==============================] - 128s 1s/step - loss: 4.2400 - accuracy: 0.8626 - val_loss: 1.2113 - val_accuracy: 0.8806
Epoch 2/50
113/113 [==============================] - 111s 986ms/step - loss: 1.0220 - accuracy: 0.9270 - val_loss: 1.0914 - val_accuracy: 0.8973
Epoch 3/50
113/113 [==============================] - 110s 972ms/step - loss: 0.7431 - accuracy: 0.9367 - val_loss: 3.6834 - val_accuracy: 0.7389
Epoch 4/50
113/113 [==============================] - 114s 1s/step - loss: 0.9173 - accuracy: 0.9284 - val_loss: 0.4548 - val_accuracy: 0.9443
Epoch 5/50
113/113 [==============================] - 102s 906ms/step - loss: 0.4801 - accuracy: 0.9447 - val_loss: 0.4900 - val_accuracy: 0.9363
Epoch 6/50
113/113 [==============================] - 99s 875ms/step - loss: 0.2916 - accuracy: 0.9519 - val_loss: 0.3769 - val_accuracy: 0.9437
Epoch 7/50
113/113 [==============================] - 99s 877ms/step - loss: 0.3990 - accuracy: 0.9405 - val_loss: 1.7419 - val_acc

## Testing a L1 Regularized Neural Network

In [15]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Define the input shape
input_shape = (150, 150, 3)

# Create the L1 regularized model
l1_model = models.Sequential()
l1_model.add(layers.Flatten(input_shape=input_shape))
l1_model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l1(0.01)))
l1_model.add(layers.Dense(1, activation='sigmoid'))
l1_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the L1 regularized model with early stopping
history_l1 = l1_model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the L1 regularized model on the validation data
val_generator.reset()
val_loss_l1, val_accuracy_l1 = l1_model.evaluate(val_generator)
print(f"L1 Regularization Model Validation Loss: {val_loss_l1}")
print(f"L1 Regularization Model Validation Accuracy: {val_accuracy_l1}")

# Get predictions for the validation data
val_preds_l1 = l1_model.predict(val_generator)
val_preds_l1 = (val_preds_l1 > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

# Calculate additional metrics
val_precision_l1 = precision_score(val_true, val_preds_l1)
val_recall_l1 = recall_score(val_true, val_preds_l1)
val_f1_l1 = f1_score(val_true, val_preds_l1)
val_conf_matrix_l1 = confusion_matrix(val_true, val_preds_l1)

print("L1 Regularization Model Validation Metrics:")
print(f"Precision: {val_precision_l1}")
print(f"Recall: {val_recall_l1}")
print(f"F1 Score: {val_f1_l1}")
print(f"Confusion Matrix:\n {val_conf_matrix_l1}")


Epoch 1/50
113/113 [==============================] - 120s 1s/step - loss: 156.8682 - accuracy: 0.6975 - val_loss: 57.5098 - val_accuracy: 0.5000
Epoch 2/50
113/113 [==============================] - 118s 1s/step - loss: 51.2147 - accuracy: 0.6869 - val_loss: 55.8907 - val_accuracy: 0.5000
Epoch 3/50
113/113 [==============================] - 117s 1s/step - loss: 44.7222 - accuracy: 0.7077 - val_loss: 40.3824 - val_accuracy: 0.5000
Epoch 4/50
113/113 [==============================] - 119s 1s/step - loss: 39.3219 - accuracy: 0.7633 - val_loss: 45.6769 - val_accuracy: 0.5000
Epoch 5/50
113/113 [==============================] - 119s 1s/step - loss: 38.9371 - accuracy: 0.7586 - val_loss: 40.5544 - val_accuracy: 0.5000
Epoch 6/50
113/113 [==============================] - 117s 1s/step - loss: 38.1817 - accuracy: 0.8128 - val_loss: 37.9983 - val_accuracy: 0.5000
Epoch 7/50
113/113 [==============================] - 120s 1s/step - loss: 38.3372 - accuracy: 0.8064 - val_loss: 38.3589 - val_a

## Testing an L2 Regularized Neural Network